In [1]:
# 1. Read Dataset
# 2. Extract valuable features
# 3. Data preprocessing
#   3.1 Remove strings from data
#   3.2 Informations are equal to each other
# 4. Divide dataset to train and test set
# 5. Create Neural Network
# 6. Compile and Fit
# 7. Let's predict
# 8. Congrats!

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F

In [2]:
dataframe = pd.read_csv("Churn_Modelling.csv")
dataframe.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataframe = dataframe.iloc[:, 3:]
dataframe.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
x = dataframe.iloc[:, :-1].values
y = dataframe.iloc[:, -1].values
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
country_encoder = LabelEncoder()
x[:, 1] = country_encoder.fit_transform(x[:, 1])
x[:, 2] = gender_encoder.fit_transform(x[:, 2])
x

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [6]:
sc = StandardScaler()
x = sc.fit_transform(x)
x


array([[-0.32622142, -0.90188624, -1.09598752, ...,  0.64609167,
         0.97024255,  0.02188649],
       [-0.44003595,  1.51506738, -1.09598752, ..., -1.54776799,
         0.97024255,  0.21653375],
       [-1.53679418, -0.90188624, -1.09598752, ...,  0.64609167,
        -1.03067011,  0.2406869 ],
       ...,
       [ 0.60498839, -0.90188624, -1.09598752, ..., -1.54776799,
         0.97024255, -1.00864308],
       [ 1.25683526,  0.30659057,  0.91241915, ...,  0.64609167,
        -1.03067011, -0.12523071],
       [ 1.46377078, -0.90188624, -1.09598752, ...,  0.64609167,
        -1.03067011, -1.07636976]])

In [7]:
x.shape

(10000, 10)

In [8]:
import torch
from torch import nn
from torch.nn import functional as F

class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(10, 32)
        self.hidden_layer1 = nn.Linear(32, 64)
        self.hidden_layer2 = nn.Linear(64, 128)
        self.hidden_layer3 = nn.Linear(128, 32)
        self.output_layer = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.hidden_layer1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.hidden_layer2(x)
        x = F.relu(x)
        if x > 0.5:
            
        x = self.hidden_layer3(x)
        x = F.relu(x)
        x = self.output_layer(x)
        x = F.sigmoid(x)      
        
        return x

In [9]:
x = torch.from_numpy(x)
y = torch.from_numpy(y).view(-1, 1)

In [10]:
dataset = torch.utils.data.TensorDataset(x, y)
train_set, test_set = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), int(0.2*len(dataset))])

In [12]:
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)


In [13]:
from torch.utils import tensorboard
model = NeuralNetwork()
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

nb_epoch = 50 
summary = tensorboard.SummaryWriter()
for n in range(nb_epoch):
    for i, data in enumerate(train_dataloader):
        input_data, label = data
        input_data = torch.autograd.Variable(input_data).float()
        label = torch.autograd.Variable(label).float()
        y_pred = model(input_data)
        optim.zero_grad()
        loss = loss_fn(label, y_pred)
        loss.backward()
        optim.step()
        summary.add_scalar("loss/train", loss.item(), i + n*len(train_dataloader))
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader):
            input_data, label = data
            input_data = torch.autograd.Variable(input_data).float()
            label = torch.autograd.Variable(label).float()
            y_pred = model(input_data)
            loss = loss_fn(label, y_pred)
            test_loss += loss.item()
    summary.add_scalar("loss/test", test_loss/len(test_dataloader), n)
    summary.flush()
    print(f"Epoch: {n} training loss: {loss.item()} test loss: {test_loss}")

C:\Users\marci\Anaconda3\lib\site-packages\torch\nn\functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0 training loss: 0.13763053715229034 test loss: 8.564240209758282
Epoch: 1 training loss: 0.1481749564409256 test loss: 7.781436197459698
Epoch: 2 training loss: 0.18692727386951447 test loss: 7.98103778809309
Epoch: 3 training loss: 0.15585961937904358 test loss: 7.484050936996937
Epoch: 4 training loss: 0.08388049155473709 test loss: 7.177475593984127
Epoch: 5 training loss: 0.13530005514621735 test loss: 7.3756221532821655
Epoch: 6 training loss: 0.05910332873463631 test loss: 7.241087097674608
Epoch: 7 training loss: 0.11431705206632614 test loss: 6.988206742331386
Epoch: 8 training loss: 0.15601730346679688 test loss: 7.19307167828083
Epoch: 9 training loss: 0.07091890275478363 test loss: 7.094670694321394
Epoch: 10 training loss: 0.14058521389961243 test loss: 7.014001615345478
Epoch: 11 training loss: 0.08228127658367157 test loss: 7.051697444170713
Epoch: 12 training loss: 0.019397353753447533 test loss: 6.959870217368007
Epoch: 13 training loss: 0.15790486335754395 test

In [ ]:
tensorboard --logdir=runs

In [15]:
with torch.no_grad():
    correct = 0
    for i, data in enumerate(test_dataloader):
        input_data, label = data
        input_data = torch.autograd.Variable(input_data).float()
        label = torch.autograd.Variable(label).float()
        y_pred = model(input_data)
        loss = loss_fn(label, y_pred)
        y_pred = torch.round(y_pred.data)
        correct += (y_pred == label).sum()

    accuracy = correct/float(len(test_dataloader)*32)
    print(f"Accuracy: {accuracy.item()}")

C:\Users\marci\Anaconda3\lib\site-packages\torch\nn\functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Accuracy: 0.8521825671195984


In [16]:
y_pred

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [17]:
label

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]])